In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

#classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style('white')  # plot formatting

# Load in Dataset
want to predict whether it will rain tmrow or not

In [2]:
df = pd.read_csv("datasets/weatherAUS.csv")
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [3]:
df.shape

(145460, 23)

In [4]:
df.isnull().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

In [5]:
df.dropna(inplace=True) #dropping any rows with nan values
df.shape

(56420, 23)

In [6]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
6049,2009-01-01,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,ENE,...,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,No,No
6050,2009-01-02,Cobar,18.4,28.9,0.0,14.8,13.0,S,37.0,SSE,...,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,No,No
6052,2009-01-04,Cobar,19.4,37.6,0.0,10.8,10.6,NNE,46.0,NNE,...,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,No,No
6053,2009-01-05,Cobar,21.9,38.4,0.0,11.4,12.2,WNW,31.0,WNW,...,37.0,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6,No,No
6054,2009-01-06,Cobar,24.2,41.0,0.0,11.2,8.4,WNW,35.0,NW,...,19.0,15.0,1010.7,1007.4,1.0,6.0,33.6,37.6,No,No


In [7]:
df.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000
mean,13.464770,24.219206,2.130397,5.503135,7.735626,40.877366,15.667228,19.786778,65.874123,49.601985,1017.239505,1014.795580,4.241705,4.326515,18.204961,22.710333
std,6.416689,6.970676,7.014822,3.696282,3.758153,13.335232,8.317005,8.510180,18.513289,20.197040,6.909357,6.870892,2.797162,2.647251,6.567991,6.836543
min,-6.700000,4.100000,0.000000,0.000000,0.000000,9.000000,2.000000,2.000000,0.000000,0.000000,980.500000,977.100000,0.000000,0.000000,-0.700000,3.700000
25%,8.600000,18.700000,0.000000,2.800000,5.000000,31.000000,9.000000,13.000000,55.000000,35.000000,1012.700000,1010.100000,1.000000,2.000000,13.100000,17.400000
50%,13.200000,23.900000,0.000000,5.000000,8.600000,39.000000,15.000000,19.000000,67.000000,50.000000,1017.200000,1014.700000,5.000000,5.000000,17.800000,22.400000
75%,18.400000,29.700000,0.600000,7.400000,10.700000,48.000000,20.000000,26.000000,79.000000,63.000000,1021.800000,1019.400000,7.000000,7.000000,23.300000,27.900000
max,31.400000,48.100000,206.200000,81.200000,14.500000,124.000000,67.000000,76.000000,100.000000,100.000000,1040.400000,1038.900000,8.000000,9.000000,39.400000,46.100000


In [8]:
df['RainTomorrow'].value_counts()

No     43993
Yes    12427
Name: RainTomorrow, dtype: int64

In [9]:
def standardizeResult(answer):
    if answer == 'Yes':
        return 1
    else:
        return 0

df['RainTomorrow'] = df['RainTomorrow'].apply(standardizeResult)

In [10]:
df['RainTomorrow'].value_counts()

0    43993
1    12427
Name: RainTomorrow, dtype: int64

In [11]:
X = df.drop(['RainTomorrow'], axis=1)
y = df['RainTomorrow']
X.shape

(56420, 22)

In [12]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56420 entries, 6049 to 142302
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           56420 non-null  object 
 1   Location       56420 non-null  object 
 2   MinTemp        56420 non-null  float64
 3   MaxTemp        56420 non-null  float64
 4   Rainfall       56420 non-null  float64
 5   Evaporation    56420 non-null  float64
 6   Sunshine       56420 non-null  float64
 7   WindGustDir    56420 non-null  object 
 8   WindGustSpeed  56420 non-null  float64
 9   WindDir9am     56420 non-null  object 
 10  WindDir3pm     56420 non-null  object 
 11  WindSpeed9am   56420 non-null  float64
 12  WindSpeed3pm   56420 non-null  float64
 13  Humidity9am    56420 non-null  float64
 14  Humidity3pm    56420 non-null  float64
 15  Pressure9am    56420 non-null  float64
 16  Pressure3pm    56420 non-null  float64
 17  Cloud9am       56420 non-null  float64
 18  Cl

In [13]:
X['Date'] = pd.to_datetime(X['Date'], 
 format = '%Y-%m-%d', 
 errors = 'coerce')

#encoding the date
X['year'] = X['Date'].dt.year
X['day'] = X['Date'].dt.day
X['month'] = X['Date'].dt.month

X.drop('Date', axis=1, inplace=True)#dropping the date
X.shape

(56420, 24)

In [14]:
#one hot encoding data
X_ohe = pd.get_dummies(X)
X_ohe

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,RainToday_No,RainToday_Yes
6049,17.9,35.2,0.0,12.0,12.3,48.0,6.0,20.0,20.0,13.0,...,0,0,0,0,1,0,0,0,1,0
6050,18.4,28.9,0.0,14.8,13.0,37.0,19.0,19.0,30.0,8.0,...,0,0,1,0,0,0,0,0,1,0
6052,19.4,37.6,0.0,10.8,10.6,46.0,30.0,15.0,42.0,22.0,...,0,0,0,0,0,0,0,0,1,0
6053,21.9,38.4,0.0,11.4,12.2,31.0,6.0,6.0,37.0,22.0,...,0,0,0,0,0,0,0,1,1,0
6054,24.2,41.0,0.0,11.2,8.4,35.0,17.0,13.0,19.0,15.0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142298,19.3,33.4,0.0,6.0,11.0,35.0,9.0,20.0,63.0,32.0,...,0,0,0,0,0,0,0,0,1,0
142299,21.2,32.6,0.0,7.6,8.6,37.0,13.0,11.0,56.0,28.0,...,0,1,0,0,0,0,0,0,1,0
142300,20.7,32.8,0.0,5.6,11.0,33.0,17.0,11.0,46.0,23.0,...,0,0,0,0,0,1,0,0,1,0
142301,19.5,31.8,0.0,6.2,10.6,26.0,9.0,17.0,62.0,58.0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
X_ohe.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,RainToday_No,RainToday_Yes
count,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,...,56420.000000,56420.000000,56420.000000,56420.000000,56420.00000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000
mean,13.464770,24.219206,2.130397,5.503135,7.735626,40.877366,15.667228,19.786778,65.874123,49.601985,...,0.072829,0.073609,0.059057,0.061769,0.07111,0.069514,0.056717,0.068345,0.779121,0.220879
std,6.416689,6.970676,7.014822,3.696282,3.758153,13.335232,8.317005,8.510180,18.513289,20.197040,...,0.259858,0.261135,0.235734,0.240737,0.25701,0.254329,0.231304,0.252338,0.414843,0.414843
min,-6.700000,4.100000,0.000000,0.000000,0.000000,9.000000,2.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.600000,18.700000,0.000000,2.800000,5.000000,31.000000,9.000000,13.000000,55.000000,35.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,13.200000,23.900000,0.000000,5.000000,8.600000,39.000000,15.000000,19.000000,67.000000,50.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,18.400000,29.700000,0.600000,7.400000,10.700000,48.000000,20.000000,26.000000,79.000000,63.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000
max,31.400000,48.100000,206.200000,81.200000,14.500000,124.000000,67.000000,76.000000,100.000000,100.000000,...,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
#standardizing data for logistic regression and KNN
scaler = StandardScaler()
scaled_columns = pd.DataFrame(scaler.fit_transform(X_ohe))
X_ohe = scaled_columns

In [17]:
#label encode for random forest
X_le = X.apply(LabelEncoder().fit_transform)
X_le.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,year,day,month
6049,4,227,283,0,120,123,11,21,1,12,...,170,177,2,5,270,280,0,2,0,0
6050,4,232,220,0,140,130,8,15,10,10,...,236,254,1,1,207,216,0,2,1,0
6052,4,242,307,0,108,106,5,20,5,6,...,230,225,1,6,291,295,0,2,3,0
6053,4,267,315,0,114,122,14,12,14,15,...,234,224,1,5,295,302,0,2,4,0
6054,4,290,341,0,112,84,14,14,7,14,...,214,207,1,6,340,322,0,2,5,0


# Algorithm 1: Logistic Regression

In [18]:
def LG_model(X, y):
    '''returns the scores of each trial'''
    
    pipe = Pipeline(steps=[('classifier', LogisticRegression())])

    search_space = [{'classifier': [LogisticRegression(max_iter=5000)],
                     'classifier__solver': ['saga'],
                     'classifier__penalty': ['l1', 'l2'],
                     'classifier__C': np.logspace(-8, 4, 13)},
                    {'classifier': [LogisticRegression(max_iter=5000)],
                     'classifier__solver': ['lbfgs'],
                     'classifier__penalty': ['l2'],
                     'classifier__C': np.logspace(-8, 4, 13)},
                    {'classifier': [LogisticRegression(max_iter=5000)],
                     'classifier__solver': ['lbfgs','saga'],
                     'classifier__penalty': ['none']}
                    ]
    
    TRIALS = 5#DOING 5 TRIALS
    log_test_acc = []#saving the accuracy for each logistic regression trial on test set
    log_test_auc = []#saving the roc auc for each logistic regression trial on test set
    log_test_f1 = []#saving the f1 score for each logistic regression trial on test set
    train_acc = []
    train_auc = []
    train_f1 = []

    for i in range(TRIALS):
        #sampling 5000 training size for k-fold
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=5000)
        
        # Create grid search |
        clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                       scoring=['accuracy', 'roc_auc', 'f1'], refit=False,
                       verbose=1, n_jobs=-1)
        best_model = clf.fit(X_train, y_train)

        #top 3 best params model parameters
        model_acc = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_accuracy'])]
        model_auc = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_roc_auc'])]
        model_f1 = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_f1'])]

        model_params = [model_acc, model_auc, model_f1]

        #gets the scores for each model and append them to corresponding array
        for index, params in enumerate(model_params):
            C = 0
            try:
                C = params['classifier__C']
            except KeyError:
               pass

            solver = params['classifier__solver']
            penalty = params['classifier__penalty']

            #training best models on validation set
            logit_model = LogisticRegression(max_iter=5000)
            logit_model.set_params(C=C, solver=solver, penalty=penalty)
            logit_model.fit(X_train, y_train) 

            #get roc_auc, acc and f1 scores on test set
            if index == 0:
                score = logit_model.score(X_test, y_test)
                log_test_acc.append(score)
                train_score = logit_model.score(X_train, y_train)
                train_acc.append(train_score)

            elif index == 1:
                roc_score = roc_auc_score(y_test, logit_model.predict_proba(X_test)[:, 1])
                log_test_auc.append(roc_score)
                train_roc = roc_auc_score(y_train, logit_model.predict_proba(X_train)[:, 1])
                train_auc.append(train_roc)

            #appending to f1
            else:
                y_predict = logit_model.predict(X_test)#predictions on test set
                f_score = f1_score(y_test, y_predict)
                log_test_f1.append(f_score)
                f_train = f1_score(y_train, logit_model.predict(X_train))
                train_f1.append(f_train)
    
    return log_test_acc, log_test_auc, log_test_f1, train_acc, train_auc, train_f1

In [19]:
log_acc, log_auc, log_f1, log_train_acc, log_train_auc, log_train_f1 = LG_model(X_ohe, y)

Fitting 5 folds for each of 41 candidates, totalling 205 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 190 out of 205 | elapsed:    9.1s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done 205 out of 205 | elapsed:    9.7s finished


Fitting 5 folds for each of 41 candidates, totalling 205 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 205 out of 205 | elapsed:   12.5s finished


Fitting 5 folds for each of 41 candidates, totalling 205 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/garvinmo/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 190 out of 205 | elapsed:   13.7s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done 205 out of 205 | elapsed:   15.0s finished


Fitting 5 folds for each of 41 candidates, totalling 205 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/garvinmo/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 190 out of 205 | elapsed:    7.8s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done 205 out of 205 | elapsed:    8.5s finished


Fitting 5 folds for each of 41 candidates, totalling 205 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  54 tasks      | elapsed:    2.1s
/Users/garvinmo/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done 190 out of 205 | elapsed:   12.9s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done 205 out of 205 | elapsed:   14.0s finished


# Algorithm 2: KNN

In [20]:
def KNN_model(X, y):
    classifier = KNeighborsClassifier()
    num_k = np.ceil(np.logspace(0, 2.7, 3)).astype(int)
    
    search_space = {'n_neighbors': num_k, 
                      'weights': ['uniform', 'distance']}

    TRIALS = 5#DOING 5 TRIALS
    acc_scores = []#saving the accuracy for each trial on test set
    auc_scores = []#saving the roc auc for each trial on test set
    f1_scores = []#saving the f1 score for each trial on test set
    train_acc = []
    train_auc = []
    train_f1 = []
    
    for i in range(TRIALS):
        #sampling 5000 training size for k-fold
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=5000)
        
        # Create grid search
        clf = GridSearchCV(classifier, search_space, cv=StratifiedKFold(n_splits=5), 
                           scoring=['accuracy', 'roc_auc', 'f1'], refit=False,
                           verbose=1, n_jobs=-1)
        best_model = clf.fit(X_train, y_train)

        #top 3 best params model parameters
        model_acc = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_accuracy'])]
        model_auc = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_roc_auc'])]
        model_f1 = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_f1'])]

        model_params = [model_acc, model_auc, model_f1]

        #gets the scores for each model and append them to corresponding array
        for index, params in enumerate(model_params):
            k = params['n_neighbors']
            weight = params['weights']

            #training best models on validation set
            KNN_model = KNeighborsClassifier()
            KNN_model.set_params(n_neighbors=k, weights=weight)
            KNN_model.fit(X_train, y_train)#training on 5000 points

            #model == model_acc
            if index == 0:
                score = KNN_model.score(X_test, y_test)
                acc_scores.append(score)
                train_score = KNN_model.score(X_train, y_train)
                train_acc.append(train_score)
                
            #model_auc
            elif index == 1:
                roc_score = roc_auc_score(y_test, KNN_model.predict_proba(X_test)[:, 1])
                auc_scores.append(roc_score)
                train_roc = roc_auc_score(y_train, KNN_model.predict_proba(X_train)[:, 1])
                train_auc.append(train_roc)

            #appending to f1
            else:
                y_predict = KNN_model.predict(X_test)#predictions on test set
                f_score = f1_score(y_test, y_predict)
                f1_scores.append(f_score)
                f_train = f1_score(y_train, KNN_model.predict(X_train))
                train_f1.append(f_train)
    
    return acc_scores, auc_scores, f1_scores, train_acc, train_auc, train_f1

In [21]:
knn_acc, knn_auc, knn_f1, knn_train_acc, knn_train_auc, knn_train_f1 = KNN_model(X_ohe, y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    7.6s finished


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    6.6s finished


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    7.1s finished


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    7.1s finished


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/garvinmo/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    7.6s finished


# Algorithm 3: Random Forest

In [22]:
def RF_model(X, y):
    classifier = RandomForestClassifier()
    max_features = [1, 2, 4, 6, 8, 12, 16, 20]
    
    for feature in max_features:
        if feature > len(X.columns):
            max_features.remove(feature)
            
    search_space = {'n_estimators': [1024], 'max_features': max_features}
    
    TRIALS = 5#DOING 5 TRIALS
    acc_scores = []#saving the accuracy for each trial on test set
    auc_scores = []#saving the roc auc for each trial on test set
    f1_scores = []#saving the f1 score for each trial on test set
    train_acc = []
    train_auc = []
    train_f1 = []
    
    for i in range(TRIALS):
        #sampling 5000 training size for k-fold
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=5000)
        
        # Create grid search
        clf = GridSearchCV(classifier, search_space, cv=StratifiedKFold(n_splits=5), 
                       scoring=['accuracy', 'roc_auc', 'f1'], refit=False,
                       verbose=2, n_jobs=-1)
        best_model = clf.fit(X_train, y_train)
        
        #top 3 best params model parameters
        model_acc = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_accuracy'])]
        model_auc = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_roc_auc'])]
        model_f1 = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_f1'])]
        
        model_params = [model_acc, model_auc, model_f1]
        
        #gets the scores for each model and append them to corresponding array
        for index, params in enumerate(model_params):
            n_estimators = params['n_estimators']
            max_features = params['max_features']
            
            #train once more on entire validation set
            RF_model = RandomForestClassifier(n_estimators=n_estimators, 
                                                  max_features=max_features)
            RF_model.fit(X_train, y_train)
            
            #scoring on test set using multiple metrics
            #model == model_acc
            if index == 0:
                score = RF_model.score(X_test, y_test)
                acc_scores.append(score)
                train_score = RF_model.score(X_train, y_train)
                train_acc.append(train_score)
            
            #model_auc
            elif index == 1:
                roc_score = roc_auc_score(y_test, RF_model.predict_proba(X_test)[:, 1])
                auc_scores.append(roc_score)
                train_roc = roc_auc_score(y_train, RF_model.predict_proba(X_train)[:, 1])
                train_auc.append(train_roc)

            #appending to f1
            else:
                y_predict = RF_model.predict(X_test)#predictions on test set
                f_score = f1_score(y_test, y_predict)
                f1_scores.append(f_score)
                f_train = f1_score(y_train, RF_model.predict(X_train))
                train_f1.append(f_train)
                
    return acc_scores, auc_scores, f1_scores, train_acc, train_auc, train_f1

In [23]:
rf_acc, rf_auc, rf_f1, rftrain_acc, rftrain_auc, rftrain_f1= RF_model(X_le, y)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/garvinmo/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   35.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.3min finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   33.5s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.3min finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/garvinmo/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   30.2s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.2min finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/garvinmo/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   31.6s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.3min finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/garvinmo/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   35.2s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.3min finished


# Results

## Logistic Regression Results:

In [24]:
print("""TEST SET PERFORMANCE:""")
scores = pd.DataFrame([log_acc, log_auc, log_f1], 
                     columns= ["Trial 1", "Trial 2", "Trial 3", "Trial 4", "Trial 5"])
scores.rename(index={0: "ACC", 1: "ROC", 2: "FSC"}, inplace=True)
print(scores)
print()

logt_scores = pd.DataFrame([log_train_acc, log_train_auc, log_train_f1], 
                     columns= ["Trial 1", "Trial 2", "Trial 3", "Trial 4", "Trial 5"])
logt_scores.rename(index={0: "ACC", 1: "ROC", 2: "FSC"}, inplace=True)
print("TRAIN SET PERFORMANCE:")
print(logt_scores)

TEST SET PERFORMANCE:
      Trial 1   Trial 2   Trial 3   Trial 4   Trial 5
ACC  0.854259  0.853092  0.853909  0.853306  0.852839
ROC  0.884416  0.884626  0.885708  0.884818  0.883683
FSC  0.624669  0.620821  0.620937  0.616581  0.623143

TRAIN SET PERFORMANCE:
      Trial 1   Trial 2   Trial 3   Trial 4   Trial 5
ACC  0.864000  0.865600  0.855400  0.853800  0.861400
ROC  0.899905  0.901048  0.889914  0.885162  0.895244
FSC  0.652785  0.636364  0.627512  0.618675  0.646392


## KNN Results:

In [25]:
print("""TEST SET PERFORMANCE:""")
k_scores = pd.DataFrame([knn_acc, knn_auc, knn_f1], 
                     columns= ["Trial 1", "Trial 2", "Trial 3", "Trial 4", "Trial 5"])
k_scores.rename(index={0: "ACC", 1: "ROC", 2: "FSC"}, inplace=True)
print(k_scores)
print()

print("TRAIN SET PERFORMANCE:")
kt_scores = pd.DataFrame([knn_train_acc, knn_train_auc, knn_train_f1], 
                     columns= ["Trial 1", "Trial 2", "Trial 3", "Trial 4", "Trial 5"])
kt_scores.rename(index={0: "ACC", 1: "ROC", 2: "FSC"}, inplace=True)
print(kt_scores)

TEST SET PERFORMANCE:
      Trial 1   Trial 2   Trial 3   Trial 4   Trial 5
ACC  0.806671  0.808401  0.805970  0.805115  0.806826
ROC  0.839923  0.831548  0.839641  0.840887  0.831302
FSC  0.415766  0.416877  0.405865  0.409531  0.415527

TRAIN SET PERFORMANCE:
     Trial 1  Trial 2  Trial 3  Trial 4  Trial 5
ACC      1.0    0.811      1.0   0.8134      1.0
ROC      1.0    1.000      1.0   1.0000      1.0
FSC      1.0    1.000      1.0   1.0000      1.0


## Random Forest Results:

In [26]:
print("TEST SET PERFORMANCE:")
rf_scores = pd.DataFrame([rf_acc, rf_auc, rf_f1], 
                     columns= ["Trial 1", "Trial 2", "Trial 3", "Trial 4", "Trial 5"])
rf_scores.rename(index={0: "ACC", 1: "ROC", 2: "FSC"}, inplace=True)
print(rf_scores)
print()
print("TRAIN SET PERFORMANCE:")
rft_scores = pd.DataFrame([rftrain_acc, rftrain_auc, rftrain_f1], 
                     columns= ["Trial 1", "Trial 2", "Trial 3", "Trial 4", "Trial 5"])
rft_scores.rename(index={0: "ACC", 1: "ROC", 2: "FSC"}, inplace=True)
print(rft_scores)

TEST SET PERFORMANCE:
      Trial 1   Trial 2   Trial 3   Trial 4   Trial 5
ACC  0.854045  0.854259  0.853656  0.853948  0.852606
ROC  0.882664  0.882683  0.879023  0.882971  0.880860
FSC  0.610177  0.610259  0.617980  0.598914  0.610656

TRAIN SET PERFORMANCE:
     Trial 1  Trial 2  Trial 3  Trial 4  Trial 5
ACC      1.0      1.0      1.0      1.0      1.0
ROC      1.0      1.0      1.0      1.0      1.0
FSC      1.0      1.0      1.0      1.0      1.0
